#Name - Chinmayee Taralkar
#Rollno - BECOB262
#HPC Assignment: Find max no, min no, mean, standard deviation in the vector

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-7chm68oh
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-7chm68oh
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=06157ff1367f4efa4f4300f4414a3f53e30ddd34cdda45d73ae71b6bf0b977d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-6_dv4j8m/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


## Find the Maximum element

In [ ]:
%%cu
#include<iostream>
#include<cstdio>

using namespace std;

__global__ void maximum(int *a,int *b,int n)
{
	int block=256*blockIdx.x;
 
	int max=0;

	for(int i=block;i<min(256+block,n);i++)
	{
		if(max<a[i])
		{
			max=a[i];	
		}
	}
	b[blockIdx.x]=max;

}

int main()
{
	int n = 512;
		int a[n];
	cudaEvent_t start,end;

	for(int i=0;i<n;i++)
	{
		a[i]=i+1;
	}

	int *ad,*bd;
	int size=n*sizeof(int);
	
	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

	int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while(n>1)
	{
		maximum<<<grids,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time=0;
	cudaEventElapsedTime(&time,start,end);
	
	int ans[2];

	cudaMemcpy(ans,ad,4,cudaMemcpyDeviceToHost);
	
	cout<<"The maximum element is "<<ans[0]<<endl;
	
	cout<<"The time required do it is ";
	cout<<time<<endl;
} 


The maximum element is 512
The time required do it is 0.036192



## Find the Minimum element


In [ ]:
%%cu
#include<iostream>
#include<cstdio>

using namespace std;

__global__ void minimum(int *a,int *b,int n)
{
	int block=256*blockIdx.x;
	int mini=7888888;

	for(int i=block;i<min(256+block,n);i++)
	{
		if(mini>a[i])
		{
			mini=a[i];
		}
	}
	b[blockIdx.x]=mini;
}

int main()
{
	int n = 512;
	int a[n];

	cudaEvent_t start,end;
	cudaEventCreate(&start);
	cudaEventCreate(&end);

	for(int i=0;i<n;i++)
	{
		a[i]=i+1;
	}
	
	int *ad,*bd;
	int size=n*sizeof(int);
	
	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

  int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);
	cudaEventRecord(start);
	while(n>1)
	{
		minimum<<<grids,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);
	float time=0;
	cudaEventElapsedTime(&time,start,end);
	
	int ans[2];

	cudaMemcpy(ans,ad,4,cudaMemcpyDeviceToHost);
	
	cout<<"The minimum element is "<<ans[0]<<endl;
	
	cout<<"The time required dor it is ";
	cout<<time<<endl;
} 


The minimum element is 1
The time required dor it is 0.06544



## Calculate the Mean

In [ ]:
%%cu
#include<iostream>
#include<cstdio>

using namespace std;

__global__ void sum(int *a,int *b,int n)
{

	int block=256*blockIdx.x;
	int sum=0;


	for(int i=block;i<min(block+256,n);i++)
	{
		sum=sum+a[i];
	}
	b[blockIdx.x]=sum;

}

int main()
{
	int n = 512;
	int a[n];
	for(int i=0;i<n;i++)
	{
		a[i]=i+1;
	}


	int *ad,*bd;
	int size=n*sizeof(int);

	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

	int grids=ceil(n*1.0f/256.0f);

	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);
	int p=n;

	cudaEvent_t start,end;
	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while(n>1)
	{

		sum<<<grid,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);

	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

  float time=0;

	cudaEventElapsedTime(&time,start,end);

  cout<<"The time required is "<<time<<endl;

	int add[2];
	n=p;

	cudaMemcpy(add,ad,4,cudaMemcpyDeviceToHost);

	cout<<"The sum is  "<<add[0]<<endl;

	float mean=0.0f;

	mean=add[0]/(n*1.0f);

	cout<<"The mean is   "<<mean<<endl;
}


The time required is 0.036384
The sum is  131328
The mean is   256.5



## Calculate the Standard Deviation

In [ ]:
%%cu

#include<iostream>
#include<cstdio>

using namespace std;


__global__ void var(int *a,int *b,int n,float mean)
{

	int block=256*blockIdx.x;
	float sum=0;
	for(int i=block;i<min(block+256,n);i++)
	{
	sum=sum+(a[i]-mean)*(a[i]-mean);
	}
	b[blockIdx.x]=sum;

}

__global__ void sum(int *a,int *b,int n)
{

	int block=256*blockIdx.x;
	int sum=0;

	for(int i=block;i<min(block+256,n);i++)
	{

	sum=sum+a[i];

	}
	b[blockIdx.x]=sum;

}

int main()
{
	int n = 512;
	int a[n];

	for(int i=0;i<n;i++)

	{

	a[i]=i+1;


	}


	int *ad,*bd;

	int size=n*sizeof(int);


	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);


	int grids=ceil(n*1.0f/256.0f);

	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);

	int p=n;

	cudaEvent_t start,end;
	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);



	while(n>1)
	{

		sum<<<grid,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);

	}


	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time=0;

	cudaEventElapsedTime(&time,start,end);

	cout<<"The time required is "<<time<<endl;

	int add[2];
	n=p;


	cudaMemcpy(add,ad,4,cudaMemcpyDeviceToHost);

	cout<<"The sum is  "<<add[0]<<endl;

	float mean=0.0f;

	mean=add[0]/(n*1.0f);

	cout<<"The mean is   "<<mean<<endl;

	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

	cudaMalloc(&bd,grids*sizeof(int));

	var<<<grid,block>>>(ad,bd,n,mean);
	n=ceil(n*1.0f/256.0f);	

	sum<<<grid,block>>>(bd,ad,n);



	cudaMemcpy(add,ad,4,cudaMemcpyDeviceToHost);

	float sd=sqrt(add[0]/p*1.0f);

	cout<<"The standard deviation is "<<sd<<endl;

}


The time required is 0.039264
The sum is  131328
The mean is   256.5
The standard deviation is 147.801

